In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.0 MB/s eta 0:00:00


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.1 MB/s eta 0:00:00


In [4]:
pip install transformers optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00


In [5]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import optuna

In [8]:
# Define your model and tokenizer
model_name = "distilbert-base-cased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)


In [9]:
# Load dataset
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/politifact_cleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=1.0, random_state=42)


# Split the dataset into training, validation, and test sets
# First, split into training and temp sets (80% training, 20% temp)
train_df, temp_df = train_test_split(df_data, test_size=0.20, random_state=42)

# Then, split the temp set into validation and test sets (50% validation, 50% test)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally,reset the index of the DataFrames
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [10]:
df.groupby("label").describe()

Unnamed: 0                                                           \
           count   mean         std    min     25%    50%     75%     max   
label                                                                       
0          432.0  839.5  124.851912  624.0  731.75  839.5  947.25  1055.0   
1          624.0  311.5  180.277564    0.0  155.75  311.5  467.25   623.0   

      word count                                                   
           count       mean       std  min  25%   50%   75%   max  
label                                                              
0          432.0  11.701389  5.222315  1.0  9.0  11.0  14.0  38.0  
1          624.0   7.931090  4.665029  1.0  5.0   7.0  10.0  53.0

In [11]:
train_df.groupby("label").describe()

Unnamed: 0                                                             \
           count     mean         std    min     25%    50%     75%     max   
label                                                                         
0          344.0  842.250  127.068042  624.0  728.75  848.0  954.25  1055.0   
1          500.0  316.502  177.314068    0.0  167.75  321.5  466.50   623.0   

      word count                                                  
           count      mean       std  min  25%   50%   75%   max  
label                                                             
0          344.0  11.56686  5.280599  1.0  9.0  11.0  14.0  38.0  
1          500.0   7.96000  4.706453  1.0  4.0   7.0  10.0  53.0

In [12]:
validation_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  805.795455  104.407426  629.0  732.75  795.5  857.50   
1           62.0  307.677419  196.936852   39.0  113.25  297.5  499.25   

              word count                                                    
          max      count       mean       std  min  25%   50%    75%   max  
label                                                                       
0      1046.0       44.0  11.795455  5.428519  2.0  9.0  12.0  13.25  37.0  
1       622.0       62.0   8.161290  5.353698  1.0  5.0   7.5   9.00  31.0

In [13]:
test_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  851.704545  123.172516  625.0  758.75  862.0  951.25   
1           62.0  274.983871  185.405227    6.0  126.50  251.5  421.75   

              word count                                                     
          max      count       mean       std  min   25%   50%    75%   max  
label                                                                        
0      1050.0       44.0  12.659091  4.513253  5.0  10.0  12.0  14.25  30.0  
1       618.0       62.0   7.467742  3.481651  1.0   5.0   7.0  10.00  16.0

In [14]:
df.head()

,Unnamed: 0,id,title,label,word count,SECTION_CLEANED
0,0,politifact14984,National Federation of Independent Business,1,5,National Federation Independent Business
1,1,politifact12944,comments in Fayetteville NC,1,4,comment Fayetteville NC
2,2,politifact333,"Romney makes pitch, hoping to close deal : Ele...",1,14,Romney make pitch hop close deal Elections The...
3,3,politifact4358,Democratic Leaders Say House Democrats Are Uni...,1,11,Democratic Leaders Say House Democrats Are Uni...
4,4,politifact779,"Budget of the United States Government, FY 2008",1,8,Budget United States Government FY 2008


In [15]:
train_texts = list(train_df['SECTION_CLEANED'])
len(train_texts)

844

In [16]:
train_labels = list(train_df['label'])
len(train_labels)

844

In [17]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
# Tokenize the dataset
train_encodings = tokenizer(train_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)


In [19]:
train_dataset = CustomDataset(train_encodings, train_labels)

In [20]:
val_texts = list(validation_df['SECTION_CLEANED'])
len(val_texts)

106

In [21]:
val_labels = list(validation_df['label'])
len(val_labels)

106

In [22]:
# Tokenize the dataset
val_encodings = tokenizer(val_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [23]:
# Define a custom dataset for validation
val_dataset = CustomDataset(val_encodings, val_labels)

In [24]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./distilbert_fake_news_classification_model",
    evaluation_strategy="steps",
    num_train_epochs=3,
    #save_total_limit=2,
    save_steps=75,
    eval_steps=75,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=75,
)

In [25]:
# Define a function for training and evaluation
def train_and_evaluate(trial):
    # Define the hyperparameters to optimize
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 15)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 1e-1)
    warmup_steps = trial.suggest_int("warmup_steps", 100, 1000)


    # Update the training arguments
    training_args.num_train_epochs = num_train_epochs
    training_args.per_device_train_batch_size = per_device_train_batch_size
    training_args.weight_decay = weight_decay
    training_args.warmup_steps = warmup_steps
    training_args.learning_rate = learning_rate

    # Create the model
    model_name = "distilbert-base-cased"
    model = DistilBertForSequenceClassification.from_pretrained(model_name)

    # Fine-tune the model
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    results = trainer.evaluate(val_dataset)


    print("Evaluation Results:", results)
    model = 45
    return results["eval_loss"]



In [26]:
# Create an Optuna study
study = optuna.create_study(direction="minimize")


[I 2023-10-18 15:07:41,458] A new study created in memory with name: no-name-f0fa1e22-d909-4b0f-ba8c-90491b18ea2e


In [27]:
# Optimize hyperparameters
study.optimize(train_and_evaluate, n_trials=15)

<ipython-input-25-e72f287baa40>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
<ipython-input-25-e72f287baa40>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 1e-1)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
75,0.691400,0.578234
150,0.436900,0.343394
225,0.251300,0.514292
300,0.142300,0.809360
375,0.129200,0.723725
450,0.025300,1.092670
525,0.006200,1.113978
600,0.001500,1.139838
675,0.000400,1.209687


[I 2023-10-18 15:09:47,581] Trial 0 finished with value: 1.2130908966064453 and parameters: {'num_train_epochs': 13, 'per_device_train_batch_size': 16, 'learning_rate': 4.564805908897983e-05, 'weight_decay': 0.0020379117301511187, 'warmup_steps': 364}. Best is trial 0 with value: 1.2130908966064453.


Evaluation Results: {'eval_loss': 1.2130908966064453, 'eval_runtime': 0.1557, 'eval_samples_per_second': 680.692, 'eval_steps_per_second': 89.903, 'epoch': 13.0}


...

Evaluation Results: {'eval_loss': 0.6501349806785583, 'eval_runtime': 0.1575, 'eval_samples_per_second': 672.9, 'eval_steps_per_second': 88.874, 'epoch': 4.0}


<ipython-input-25-e72f287baa40>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
<ipython-input-25-e72f287baa40>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 1e-1)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
75,0.685800,0.672047
150,0.616000,0.460740
225,0.437300,0.518302
300,0.513700,0.532233
375,0.533600,0.463134
450,0.401500,0.488860
525,0.325600,0.542484
600,0.280700,0.764404
675,0.241000,0.877650
750,0.361700,1.030424


[I 2023-10-18 15:27:12,496] Trial 14 finished with value: 1.1538578271865845 and parameters: {'num_train_epochs': 8, 'per_device_train_batch_size': 4, 'learning_rate': 4.62744935583222e-05, 'weight_decay': 0.002439657836124098, 'warmup_steps': 674}. Best is trial 7 with value: 0.3953818082809448.


Evaluation Results: {'eval_loss': 1.1538578271865845, 'eval_runtime': 0.1557, 'eval_samples_per_second': 680.691, 'eval_steps_per_second': 89.903, 'epoch': 8.0}


In [28]:
# Get the best hyperparameters
best_params = study.best_params
best_num_train_epochs = best_params["num_train_epochs"]
best_batch_size = best_params["per_device_train_batch_size"]
best_learning_rate = best_params["learning_rate"]
best_weight_decay = best_params["weight_decay"]
best_warmup_steps = best_params["warmup_steps"]
# Print the best hyperparameters
print(f"Best Number of Epochs: {best_num_train_epochs}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Weight Decay: {best_weight_decay}")
print(f"Best Warmup Steps: {best_warmup_steps}")

Best Number of Epochs: 3
Best Batch Size: 16
Best Learning Rate: 2.5822557740619007e-05
Best Weight Decay: 0.005780473477778428
Best Warmup Steps: 399
